In [26]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully")


Libraries imported successfully


In [27]:
# Load JSON files
print("Loading JSON files...")

with open('catastro_parcels.json', 'r', encoding='utf-8') as f:
    parcels_data = json.load(f)
    
with open('catastro_buildings.json', 'r', encoding='utf-8') as f:
    buildings_data = json.load(f)
    
with open('catastro_units.json', 'r', encoding='utf-8') as f:
    units_data = json.load(f)

print(f"Loaded {len(parcels_data)} parcels")
print(f"Loaded {len(buildings_data)} buildings")
print(f"Loaded {len(units_data)} units")


Loading JSON files...
Loaded 10110 parcels
Loaded 6495 buildings
Loaded 7924 units


In [28]:
# Examine structure of first few records
print("PARCELS - Sample structure:")
if parcels_data:
    print("Keys:", list(parcels_data[0].keys()))
    print("Sample record:")
    for key, value in list(parcels_data[0].items())[:5]:
        print(f"  {key}: {value}")
print("\n" + "="*50 + "\n")

print("BUILDINGS - Sample structure:")
if buildings_data:
    print("Keys:", list(buildings_data[0].keys()))
    print("Sample record:")
    for key, value in list(buildings_data[0].items())[:5]:
        print(f"  {key}: {value}")
print("\n" + "="*50 + "\n")

print("UNITS - Sample structure:")
if units_data:
    print("Keys:", list(units_data[0].keys()))
    print("Sample record:")
    for key, value in list(units_data[0].items())[:5]:
        print(f"  {key}: {value}")


PARCELS - Sample structure:
Keys: ['id', 'referencia_catastral', 'municipio', 'codigo_municipio', 'provincia', 'codigo_provincia', 'superficie_parcela', 'uso_parcela', 'geometry', 'last_update']
Sample record:
  id: 2872
  referencia_catastral: 000100100DD99E
  municipio: SELVA
  codigo_municipio: 07058
  provincia: Illes Balears


BUILDINGS - Sample structure:
Keys: ['id', 'parcel_ref', 'building_type', 'description', 'built_area', 'staircase', 'floor', 'door', 'municipality', 'province', 'last_update']
Sample record:
  id: 1
  parcel_ref: 000100200DD89A
  building_type: VIVIENDA
  description: VIVIENDA UNIFAMILIAR
  built_area: 22


UNITS - Sample structure:
Keys: ['id', 'parcel_ref', 'unit_ref', 'car', 'cc1', 'cc2', 'province_code', 'municipality_code', 'cadastral_municipio', 'use_type', 'floor_area', 'year_built', 'participation', 'street_name', 'street_type', 'street_code', 'portal_number', 'portal_suffix', 'floor', 'door', 'staircase', 'postal_code', 'address_code', 'local_zoning

In [29]:
# Convert to DataFrames
df_parcels = pd.DataFrame(parcels_data)
df_buildings = pd.DataFrame(buildings_data)
df_units = pd.DataFrame(units_data)

print(f"Parcels DataFrame shape: {df_parcels.shape}")
print(f"Buildings DataFrame shape: {df_buildings.shape}")
print(f"Units DataFrame shape: {df_units.shape}")

# Check for referencia_catastral columns
print("\nColumns containing 'referencia' or 'catastral':")
for df_name, df in [('Parcels', df_parcels), ('Buildings', df_buildings), ('Units', df_units)]:
    ref_cols = [col for col in df.columns if 'referencia' in col.lower() or 'catastral' in col.lower()]
    print(f"{df_name}: {ref_cols}")


Parcels DataFrame shape: (10110, 10)
Buildings DataFrame shape: (6495, 11)
Units DataFrame shape: (7924, 39)

Columns containing 'referencia' or 'catastral':
Parcels: ['referencia_catastral']
Buildings: []
Units: []


In [30]:
# Define the correct column mappings based on the data structure
parcel_ref_col = 'referencia_catastral'  # Unique identifier in parcels
building_ref_col = 'parcel_ref'          # Reference to parcels in buildings
unit_ref_col = 'parcel_ref'              # Reference to parcels in units

print(f"Using correct column mappings:")
print(f"Parcels ID column: {parcel_ref_col}")
print(f"Buildings reference column: {building_ref_col}")
print(f"Units reference column: {unit_ref_col}")

# Verify the columns exist
print(f"\nColumn verification:")
print(f"'{parcel_ref_col}' in parcels: {parcel_ref_col in df_parcels.columns}")
print(f"'{building_ref_col}' in buildings: {building_ref_col in df_buildings.columns}")
print(f"'{unit_ref_col}' in units: {unit_ref_col in df_units.columns}")

# Show key columns available
print(f"\nKey columns found:")
print(f"Parcels: {[col for col in df_parcels.columns if any(term in col.lower() for term in ['referencia', 'superficie', 'municipio', 'provincia'])]}")
print(f"Buildings: {[col for col in df_buildings.columns if any(term in col.lower() for term in ['parcel_ref', 'built_area', 'building_type'])]}")
print(f"Units: {[col for col in df_units.columns if any(term in col.lower() for term in ['parcel_ref', 'floor_area', 'use_type', 'year_built'])]}")


Using correct column mappings:
Parcels ID column: referencia_catastral
Buildings reference column: parcel_ref
Units reference column: parcel_ref

Column verification:
'referencia_catastral' in parcels: True
'parcel_ref' in buildings: True
'parcel_ref' in units: True

Key columns found:
Parcels: ['referencia_catastral', 'municipio', 'codigo_municipio', 'provincia', 'codigo_provincia', 'superficie_parcela']
Buildings: ['parcel_ref', 'building_type', 'built_area']
Units: ['parcel_ref', 'use_type', 'floor_area', 'year_built']


In [31]:
# Check uniqueness of referencia_catastral in parcels
if parcel_ref_col:
    unique_parcels = df_parcels[parcel_ref_col].nunique()
    total_parcels = len(df_parcels)
    print(f"Parcels - Unique {parcel_ref_col}: {unique_parcels}, Total records: {total_parcels}")
    print(f"Is unique: {unique_parcels == total_parcels}")
    
    # Show sample values
    print(f"\nSample {parcel_ref_col} values:")
    print(df_parcels[parcel_ref_col].head(10).tolist())


Parcels - Unique referencia_catastral: 10110, Total records: 10110
Is unique: True

Sample referencia_catastral values:
['000100100DD99E', '000100100DE90A', '000100100DE90B', '000100200DE90A', '000100200DE90B', '000100300DD99G', '000100300DE90B', '000100400DD99G', '000100400DE90B', '000100500DD99G']


In [32]:
# Aggregate buildings data by parcel
print("Processing buildings data...")

# Create building aggregations
building_agg = df_buildings.groupby(building_ref_col).agg({
    'id': 'count',  # Count of buildings per parcel
    'built_area': ['sum', 'mean', 'max'],  # Built area statistics
}).round(2)

# Flatten column names
building_agg.columns = ['num_buildings', 'total_built_area', 'avg_built_area', 'max_built_area']

# Add individual building details - structured approach for unlimited buildings
def process_buildings_for_parcel(group):
    # Lists for CSV format
    built_areas = []
    building_types = []
    floors = []
    descriptions = []
    
    # Detailed dict for JSON format
    building_details = []
    
    for idx, building in group.iterrows():
        built_area = building['built_area'] if pd.notna(building['built_area']) else 0
        building_type = building['building_type'] if pd.notna(building['building_type']) else 'Unknown'
        floor = building['floor'] if pd.notna(building['floor']) else 'Unknown'
        description = building['description'] if pd.notna(building['description']) else 'Unknown'
        staircase = building['staircase'] if pd.notna(building['staircase']) else 'Unknown'
        door = building['door'] if pd.notna(building['door']) else 'Unknown'
        
        # Add to lists (for CSV columns)
        built_areas.append(built_area)
        building_types.append(building_type)
        floors.append(floor)
        descriptions.append(description)
        
        # Add to detailed structure (for JSON column)
        building_details.append({
            'building_id': int(building['id']),
            'built_area': built_area,
            'building_type': building_type,
            'description': description,
            'floor': floor,
            'staircase': staircase,
            'door': door
        })
    
    return {
        'areas_csv': ', '.join(map(str, built_areas)),
        'types_csv': ', '.join(building_types),
        'floors_csv': ', '.join(floors),
        'descriptions_csv': ', '.join(descriptions),
        'buildings_details_json': json.dumps(building_details, ensure_ascii=False)
    }

# Process all buildings for each parcel
buildings_processed = df_buildings.groupby(building_ref_col).apply(process_buildings_for_parcel)

# Add the structured building data to aggregations
building_agg['buildings_areas'] = buildings_processed.apply(lambda x: x['areas_csv'])
building_agg['buildings_types'] = buildings_processed.apply(lambda x: x['types_csv'])
building_agg['buildings_floors'] = buildings_processed.apply(lambda x: x['floors_csv'])
building_agg['buildings_descriptions'] = buildings_processed.apply(lambda x: x['descriptions_csv'])
building_agg['buildings_details_json'] = buildings_processed.apply(lambda x: x['buildings_details_json'])

# Add building type diversity
building_type_counts = df_buildings.groupby(building_ref_col)['building_type'].agg(['nunique', 'count'])
building_agg['unique_building_types'] = building_type_counts['nunique']
building_agg['building_type_diversity'] = building_type_counts['nunique'] / building_type_counts['count']

# Most common building type per parcel
most_common_type = df_buildings.groupby(building_ref_col)['building_type'].agg(lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Unknown')
building_agg['primary_building_type'] = most_common_type

building_agg = building_agg.reset_index()
print(f"Building aggregations shape: {building_agg.shape}")
print(f"Building aggregations columns: {building_agg.columns.tolist()}")
print(f"Sample building aggregation (showing structured building details):")
if len(building_agg) > 0:
    # Show columns that include structured building info
    sample_cols = [building_ref_col, 'num_buildings', 'buildings_areas', 'buildings_types', 'total_built_area']
    available_sample_cols = [col for col in sample_cols if col in building_agg.columns]
    print(building_agg[available_sample_cols].head(3))


Processing buildings data...
Building aggregations shape: (1982, 13)
Building aggregations columns: ['parcel_ref', 'num_buildings', 'total_built_area', 'avg_built_area', 'max_built_area', 'buildings_areas', 'buildings_types', 'buildings_floors', 'buildings_descriptions', 'buildings_details_json', 'unique_building_types', 'building_type_diversity', 'primary_building_type']
Sample building aggregation (showing structured building details):
       parcel_ref  num_buildings     buildings_areas  \
0  000100100DD88G              4    179, 145, 38, 42   
1  000100200DD88G              5  47, 92, 41, 27, 46   
2  000100200DD89A              1                  22   

                                     buildings_types  total_built_area  
0         VIVIENDA, VIVIENDA, PORCHE 100%, DEPORTIVO               404  
1  VIVIENDA, VIVIENDA, DEPORTIVO, ALMACEN, PORCHE...               253  
2                                           VIVIENDA                22  


In [33]:
# Aggregate units data by parcel
print("Processing units data...")

# Create unit aggregations
unit_agg = df_units.groupby(unit_ref_col).agg({
    'id': 'count',  # Count of units per parcel
    'floor_area': ['sum', 'mean', 'max', 'min'],  # Floor area statistics
    'year_built': ['mean', 'min', 'max'],  # Building year statistics
    'participation': ['sum', 'mean'],  # Participation statistics
}).round(2)

# Flatten column names
unit_agg.columns = [
    'num_units', 
    'total_floor_area', 'avg_floor_area', 'max_floor_area', 'min_floor_area',
    'avg_year_built', 'oldest_year_built', 'newest_year_built',
    'total_participation', 'avg_participation'
]

# Add individual unit details - structured approach for unlimited units
def process_units_for_parcel(group):
    current_year = datetime.now().year
    
    # Lists for CSV format
    years_built = []
    ages = []
    floor_areas = []
    use_types = []
    
    # Detailed dict for JSON format
    unit_details = []
    
    for idx, unit in group.iterrows():
        year_built = unit['year_built'] if pd.notna(unit['year_built']) else None
        age = current_year - year_built if year_built is not None else None
        floor_area = unit['floor_area'] if pd.notna(unit['floor_area']) else 0
        use_type = unit['use_type'] if pd.notna(unit['use_type']) else 'Unknown'
        
        # Add to lists (for CSV columns)
        if year_built is not None:
            years_built.append(int(year_built))
            ages.append(int(age))
        floor_areas.append(floor_area)
        use_types.append(use_type)
        
        # Add to detailed structure (for JSON column)
        unit_details.append({
            'unit_id': int(unit['id']),
            'year_built': int(year_built) if year_built is not None else None,
            'age': int(age) if age is not None else None,
            'floor_area': floor_area,
            'use_type': use_type,
            'participation': unit['participation'] if pd.notna(unit['participation']) else 0
        })
    
    return {
        'years_built_csv': ', '.join(map(str, years_built)),
        'ages_csv': ', '.join(map(str, ages)),
        'floor_areas_csv': ', '.join(map(str, floor_areas)),
        'use_types_csv': ', '.join(use_types),
        'units_details_json': json.dumps(unit_details, ensure_ascii=False)
    }

# Process all units for each parcel
units_processed = df_units.groupby(unit_ref_col).apply(process_units_for_parcel)

# Add the structured unit data to aggregations
unit_agg['units_years_built'] = units_processed.apply(lambda x: x['years_built_csv'])
unit_agg['units_ages'] = units_processed.apply(lambda x: x['ages_csv'])
unit_agg['units_floor_areas'] = units_processed.apply(lambda x: x['floor_areas_csv'])
unit_agg['units_use_types'] = units_processed.apply(lambda x: x['use_types_csv'])
unit_agg['units_details_json'] = units_processed.apply(lambda x: x['units_details_json'])

# Add use type diversity and statistics
use_type_stats = df_units.groupby(unit_ref_col)['use_type'].agg(['nunique', 'count'])
unit_agg['unique_use_types'] = use_type_stats['nunique']
unit_agg['use_type_diversity'] = use_type_stats['nunique'] / use_type_stats['count']

# Most common use type per parcel
most_common_use = df_units.groupby(unit_ref_col)['use_type'].agg(lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Unknown')
unit_agg['primary_use_type'] = most_common_use

# Calculate residential vs non-residential ratios
def calc_residential_ratio(group):
    total = len(group)
    residential = (group == 'Residencial').sum()
    return residential / total if total > 0 else 0

residential_ratio = df_units.groupby(unit_ref_col)['use_type'].apply(calc_residential_ratio)
unit_agg['residential_ratio'] = residential_ratio

# Province and municipality (taking first occurrence per parcel)
location_info = df_units.groupby(unit_ref_col)[['province', 'municipality']].first()
unit_agg['province'] = location_info['province']
unit_agg['municipality'] = location_info['municipality']

unit_agg = unit_agg.reset_index()
print(f"Unit aggregations shape: {unit_agg.shape}")
print(f"Unit aggregations columns: {unit_agg.columns.tolist()}")
print(f"Sample unit aggregation (showing structured unit details):")
if len(unit_agg) > 0:
    # Show columns that include structured unit info
    sample_cols = [unit_ref_col, 'num_units', 'units_years_built', 'units_ages', 'units_use_types', 'total_floor_area']
    available_sample_cols = [col for col in sample_cols if col in unit_agg.columns]
    print(unit_agg[available_sample_cols].head(3))


Processing units data...
Unit aggregations shape: (4977, 22)
Unit aggregations columns: ['parcel_ref', 'num_units', 'total_floor_area', 'avg_floor_area', 'max_floor_area', 'min_floor_area', 'avg_year_built', 'oldest_year_built', 'newest_year_built', 'total_participation', 'avg_participation', 'units_years_built', 'units_ages', 'units_floor_areas', 'units_use_types', 'units_details_json', 'unique_use_types', 'use_type_diversity', 'primary_use_type', 'residential_ratio', 'province', 'municipality']
Sample unit aggregation (showing structured unit details):
       parcel_ref  num_units units_years_built units_ages units_use_types  \
0  000100100DD88G          1              1940         85     Residencial   
1  000100100DD99E          1              1970         55     Residencial   
2  000100100DE90A          1              1980         45      Industrial   

   total_floor_area  
0             404.0  
1             179.0  
2               4.0  


In [34]:
# Start with parcels as the base
final_df = df_parcels.copy()
print(f"Starting with parcels: {final_df.shape}")

# Merge buildings aggregations
print("Merging buildings data...")
final_df = final_df.merge(building_agg, left_on=parcel_ref_col, right_on=building_ref_col, how='left')
print(f"After merging buildings: {final_df.shape}")
    
# Merge units aggregations  
print("Merging units data...")
final_df = final_df.merge(unit_agg, left_on=parcel_ref_col, right_on=unit_ref_col, how='left')
print(f"After merging units: {final_df.shape}")

# Fill NaN values for count and numeric columns with appropriate defaults
count_columns = [col for col in final_df.columns if col.startswith('num_')]
area_columns = [col for col in final_df.columns if 'area' in col.lower() and col.startswith(('total_', 'avg_', 'max_', 'min_'))]
ratio_columns = [col for col in final_df.columns if any(term in col.lower() for term in ['ratio', 'diversity'])]

final_df[count_columns] = final_df[count_columns].fillna(0)
final_df[area_columns] = final_df[area_columns].fillna(0)  
final_df[ratio_columns] = final_df[ratio_columns].fillna(0)

# Fill categorical columns with appropriate defaults
categorical_fill_values = {
    'primary_building_type': 'No Buildings',
    'primary_use_type': 'No Units',
    'unique_building_types': 0,
    'unique_use_types': 0
}

for col, fill_value in categorical_fill_values.items():
    if col in final_df.columns:
        final_df[col] = final_df[col].fillna(fill_value)

print(f"\nFinal DataFrame shape: {final_df.shape}")
print(f"Final DataFrame columns: {len(final_df.columns)}")

# Show a sample of merged data with structured details
print(f"\nSample of merged data with structured building/unit details:")
key_display_cols = [parcel_ref_col, 'superficie_parcela', 'num_buildings', 'num_units', 'units_years_built', 'buildings_areas']
available_cols = [col for col in key_display_cols if col in final_df.columns]
if available_cols:
    print(final_df[available_cols].head(3))
    
# Show example of how to use the structured data
print(f"\n💡 Example of how to use structured data:")
if 'units_years_built' in final_df.columns and len(final_df) > 0:
    sample_row = final_df.iloc[0]
    if pd.notna(sample_row['units_years_built']) and sample_row['units_years_built'] != '':
        years_list = sample_row['units_years_built'].split(', ')
        print(f"   Parcel {sample_row[parcel_ref_col]} has units built in: {years_list}")
        print(f"   In your code: years = df['units_years_built'].str.split(', ')")
    
    if 'units_details_json' in final_df.columns and pd.notna(sample_row['units_details_json']):
        try:
            units_json = json.loads(sample_row['units_details_json'])
            if units_json:
                print(f"   First unit details: {units_json[0]}")
                print(f"   In your code: import json; units = json.loads(df['units_details_json'].iloc[0])")
        except:
            pass


Starting with parcels: (10110, 10)
Merging buildings data...
After merging buildings: (10110, 23)
Merging units data...
After merging units: (10110, 45)

Final DataFrame shape: (10110, 45)
Final DataFrame columns: 45

Sample of merged data with structured building/unit details:
  referencia_catastral  superficie_parcela  num_buildings  num_units  \
0       000100100DD99E              252.55            0.0        1.0   
1       000100100DE90A                4.03            0.0        1.0   
2       000100100DE90B               88.54            0.0        1.0   

  units_years_built buildings_areas  
0              1970             NaN  
1              1980             NaN  
2              1987             NaN  

💡 Example of how to use structured data:
   Parcel 000100100DD99E has units built in: ['1970']
   In your code: years = df['units_years_built'].str.split(', ')
   First unit details: {'unit_id': 4, 'year_built': 1970, 'age': 55, 'floor_area': 179.0, 'use_type': 'Residencial', 'p

In [35]:
# Add additional computed metrics
print("Computing additional metrics...")

# Building density (buildings per parcel area)
if 'num_buildings' in final_df.columns and 'superficie_parcela' in final_df.columns:
    final_df['building_density_per_sqm'] = final_df['num_buildings'] / (final_df['superficie_parcela'] + 1e-6)
    print("✓ Added building density per sqm")

# Unit density (units per parcel area)  
if 'num_units' in final_df.columns and 'superficie_parcela' in final_df.columns:
    final_df['unit_density_per_sqm'] = final_df['num_units'] / (final_df['superficie_parcela'] + 1e-6)
    print("✓ Added unit density per sqm")

# Average units per building
if 'num_units' in final_df.columns and 'num_buildings' in final_df.columns:
    final_df['avg_units_per_building'] = final_df['num_units'] / (final_df['num_buildings'] + 1e-6)
    final_df['avg_units_per_building'] = final_df['avg_units_per_building'].replace([np.inf, -np.inf], 0)
    print("✓ Added average units per building")

# Development intensity (built area vs parcel area)
if 'total_built_area' in final_df.columns and 'superficie_parcela' in final_df.columns:
    final_df['building_coverage_ratio'] = final_df['total_built_area'] / (final_df['superficie_parcela'] + 1e-6)
    final_df['building_coverage_ratio'] = final_df['building_coverage_ratio'].replace([np.inf, -np.inf], 0)
    print("✓ Added building coverage ratio")

# Floor area ratio (total floor area vs parcel area)
if 'total_floor_area' in final_df.columns and 'superficie_parcela' in final_df.columns:
    final_df['floor_area_ratio'] = final_df['total_floor_area'] / (final_df['superficie_parcela'] + 1e-6)
    final_df['floor_area_ratio'] = final_df['floor_area_ratio'].replace([np.inf, -np.inf], 0)
    print("✓ Added floor area ratio (FAR)")

# Combined total constructed area (buildings + units)
if 'total_built_area' in final_df.columns and 'total_floor_area' in final_df.columns:
    final_df['total_constructed_area'] = final_df['total_built_area'] + final_df['total_floor_area']
    print("✓ Added total constructed area")

# Development age metrics
if 'avg_year_built' in final_df.columns:
    current_year = datetime.now().year
    final_df['avg_building_age'] = current_year - final_df['avg_year_built']
    final_df['avg_building_age'] = final_df['avg_building_age'].fillna(0)
    print("✓ Added average building age")

# Property value indicators (based on area, age, type)
# Efficiency ratio (floor area per building area)
if 'total_floor_area' in final_df.columns and 'total_built_area' in final_df.columns:
    final_df['area_efficiency_ratio'] = final_df['total_floor_area'] / (final_df['total_built_area'] + 1e-6)
    final_df['area_efficiency_ratio'] = final_df['area_efficiency_ratio'].replace([np.inf, -np.inf], 0)
    print("✓ Added area efficiency ratio")

# Parcel utilization score (0-1 scale)
if all(col in final_df.columns for col in ['building_coverage_ratio', 'floor_area_ratio', 'num_buildings', 'superficie_parcela']):
    # Normalize factors for scoring
    coverage_norm = np.clip(final_df['building_coverage_ratio'], 0, 1)
    far_norm = np.clip(final_df['floor_area_ratio'] / 3, 0, 1)  # FAR of 3 = max score
    density_norm = np.clip(final_df['building_density_per_sqm'] * 100, 0, 1)  # Normalize density
    
    final_df['utilization_score'] = (coverage_norm * 0.4 + far_norm * 0.4 + density_norm * 0.2).round(3)
    print("✓ Added parcel utilization score")

print(f"\nFinal DataFrame with computed metrics shape: {final_df.shape}")

# Show summary of new computed columns
computed_cols = [col for col in final_df.columns if any(term in col for term in 
                ['density', 'ratio', 'efficiency', 'score', 'age', 'constructed'])]
print(f"Computed metrics added: {len(computed_cols)}")
print(f"Computed columns: {computed_cols}")


Computing additional metrics...
✓ Added building density per sqm
✓ Added unit density per sqm
✓ Added average units per building
✓ Added building coverage ratio
✓ Added floor area ratio (FAR)
✓ Added total constructed area
✓ Added average building age
✓ Added area efficiency ratio
✓ Added parcel utilization score

Final DataFrame with computed metrics shape: (10110, 54)
Computed metrics added: 10
Computed columns: ['units_ages', 'residential_ratio', 'building_density_per_sqm', 'unit_density_per_sqm', 'building_coverage_ratio', 'floor_area_ratio', 'total_constructed_area', 'avg_building_age', 'area_efficiency_ratio', 'utilization_score']


In [36]:
# Export to Excel with multiple sheets
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
excel_filename = f'catastro_comprehensive_data_{timestamp}.xlsx'

print(f"Exporting to {excel_filename}...")

with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
    # Main comprehensive sheet
    final_df.to_excel(writer, sheet_name='Comprehensive_Data', index=False)
    
    # Key metrics sheet for search/analysis (most important columns)
    key_cols = [
        parcel_ref_col, 'superficie_parcela', 'municipio', 'provincia',
        'num_buildings', 'num_units', 
        'total_built_area', 'total_floor_area', 'total_constructed_area',
        'primary_building_type', 'primary_use_type', 'residential_ratio',
        'avg_year_built', 'avg_building_age',
        'building_density_per_sqm', 'unit_density_per_sqm', 'floor_area_ratio',
        'building_coverage_ratio', 'utilization_score',
        # Add structured details to search sheet for easy filtering
        'units_years_built', 'units_ages', 'units_use_types',
        'buildings_areas', 'buildings_types'
    ]
    available_key_cols = [col for col in key_cols if col in final_df.columns]
    final_df[available_key_cols].to_excel(writer, sheet_name='Search_Ready_Data', index=False)
    
    # Detailed Building & Unit Info sheet (with structured data)
    detail_cols = [
        parcel_ref_col, 'superficie_parcela', 'num_buildings', 'num_units',
        # Building structured data
        'buildings_areas', 'buildings_types', 'buildings_floors', 'buildings_descriptions',
        'buildings_details_json',
        # Unit structured data  
        'units_years_built', 'units_ages', 'units_floor_areas', 'units_use_types',
        'units_details_json',
        # Summary stats
        'total_built_area', 'total_floor_area', 'avg_year_built'
    ]
    available_detail_cols = [col for col in detail_cols if col in final_df.columns]
    final_df[available_detail_cols].to_excel(writer, sheet_name='Detailed_Properties', index=False)
    
    # Building statistics summary
    building_summary = final_df.groupby('primary_building_type').agg({
        'num_buildings': ['count', 'sum', 'mean'],
        'total_built_area': ['sum', 'mean'],
        'avg_built_area': 'mean',
        'superficie_parcela': 'mean'
    }).round(2)
    building_summary.columns = ['Parcels_Count', 'Total_Buildings', 'Avg_Buildings_per_Parcel', 
                               'Total_Built_Area', 'Avg_Built_Area_per_Parcel', 'Avg_Built_Area_per_Building', 'Avg_Parcel_Size']
    building_summary.to_excel(writer, sheet_name='Building_Summary')
    
    # Usage type analysis
    usage_summary = final_df.groupby('primary_use_type').agg({
        'num_units': ['count', 'sum', 'mean'],
        'total_floor_area': ['sum', 'mean'],
        'residential_ratio': 'mean',
        'avg_year_built': 'mean'
    }).round(2)
    usage_summary.columns = ['Parcels_Count', 'Total_Units', 'Avg_Units_per_Parcel',
                            'Total_Floor_Area', 'Avg_Floor_Area_per_Parcel', 'Avg_Residential_Ratio', 'Avg_Year_Built']
    usage_summary.to_excel(writer, sheet_name='Usage_Summary')
    
    # Municipality analysis
    municipality_summary = final_df.groupby('municipio').agg({
        parcel_ref_col: 'count',
        'num_buildings': 'sum',
        'num_units': 'sum', 
        'superficie_parcela': ['sum', 'mean'],
        'total_constructed_area': 'sum',
        'utilization_score': 'mean'
    }).round(2)
    municipality_summary.columns = ['Total_Parcels', 'Total_Buildings', 'Total_Units', 
                                   'Total_Land_Area', 'Avg_Parcel_Size', 'Total_Constructed_Area', 'Avg_Utilization_Score']
    municipality_summary.to_excel(writer, sheet_name='Municipality_Summary')
    
    # Summary statistics sheet
    numeric_cols = final_df.select_dtypes(include=[np.number]).columns
    summary_stats = final_df[numeric_cols].describe()
    summary_stats.to_excel(writer, sheet_name='Statistical_Summary')
    
    # Column information sheet (handle complex data types safely)
    def safe_nunique(series):
        try:
            return series.nunique()
        except (TypeError, ValueError):
            # For unhashable types like dicts, count non-null values instead
            return series.notna().sum()
    
    def safe_sample_value(series):
        try:
            if len(series) > 0 and series.notna().any():
                sample_val = series.dropna().iloc[0]
                # Truncate very long values
                str_val = str(sample_val)
                return str_val[:100] + "..." if len(str_val) > 100 else str_val
            return ''
        except:
            return 'Complex data type'
    
    col_info = pd.DataFrame({
        'Column_Name': final_df.columns,
        'Data_Type': [str(final_df[col].dtype) for col in final_df.columns],
        'Non_Null_Count': [final_df[col].notna().sum() for col in final_df.columns],
        'Null_Percentage': [round(final_df[col].isna().sum() / len(final_df) * 100, 2) for col in final_df.columns],
        'Unique_Values': [safe_nunique(final_df[col]) for col in final_df.columns],
        'Sample_Value': [safe_sample_value(final_df[col]) for col in final_df.columns]
    })
    col_info.to_excel(writer, sheet_name='Column_Info', index=False)

print(f"✅ Successfully exported to {excel_filename}")
print(f"📊 File contains {len(final_df):,} rows and {len(final_df.columns)} columns")
print(f"📁 File size: {round(sum(final_df.memory_usage(deep=True)) / 1024**2, 2)} MB")

print("\n📋 Sheets created:")
print("  • Comprehensive_Data: Complete dataset with all columns")
print("  • Search_Ready_Data: Key columns optimized for search functionality")
print("  • Detailed_Properties: Structured building and unit details (CSV + JSON)")
print("  • Building_Summary: Statistics grouped by building type")
print("  • Usage_Summary: Statistics grouped by usage type")
print("  • Municipality_Summary: Statistics grouped by municipality")
print("  • Statistical_Summary: Descriptive statistics for numeric columns")
print("  • Column_Info: Metadata about each column")

print(f"\n🎯 Perfect for your search system! Use 'Search_Ready_Data' sheet for:")
print("  • Filtering by region (municipio, provincia)")
print("  • Area-based searches (superficie_parcela, total areas)")
print("  • Usage type filtering (primary_use_type, residential_ratio)")  
print("  • Development intensity (utilization_score, coverage ratios)")
print("  • Building characteristics (age, density, types)")
print("  • Individual building/unit data (units_years_built, units_ages)")

print(f"\n🏠 Structured Data Features (UNLIMITED buildings/units per parcel!):")
print("  • CSV format: units_years_built = '1970, 1980, 1990'")
print("  • CSV format: units_ages = '54, 44, 34'")
print("  • CSV format: buildings_areas = '120, 80, 200'")
print("  • JSON format: units_details_json = complete unit info with IDs")
print("  • JSON format: buildings_details_json = complete building info with IDs")
print("  • No limits on number of buildings or units per parcel!")
print("  • Easy to parse in your search system - split by comma or parse JSON")


Exporting to catastro_comprehensive_data_20250701_120602.xlsx...
✅ Successfully exported to catastro_comprehensive_data_20250701_120602.xlsx
📊 File contains 10,110 rows and 54 columns
📁 File size: 17.75 MB

📋 Sheets created:
  • Comprehensive_Data: Complete dataset with all columns
  • Search_Ready_Data: Key columns optimized for search functionality
  • Detailed_Properties: Structured building and unit details (CSV + JSON)
  • Building_Summary: Statistics grouped by building type
  • Usage_Summary: Statistics grouped by usage type
  • Municipality_Summary: Statistics grouped by municipality
  • Statistical_Summary: Descriptive statistics for numeric columns
  • Column_Info: Metadata about each column

🎯 Perfect for your search system! Use 'Search_Ready_Data' sheet for:
  • Filtering by region (municipio, provincia)
  • Area-based searches (superficie_parcela, total areas)
  • Usage type filtering (primary_use_type, residential_ratio)
  • Development intensity (utilization_score, cover

In [37]:
# Display comprehensive data summary
print("=" * 60)
print("🏗️  CATASTRO DATA PROCESSING COMPLETE")
print("=" * 60)

print(f"\n📊 DATASET OVERVIEW:")
print(f"   • Total parcels processed: {len(final_df):,}")
print(f"   • Total columns created: {len(final_df.columns)}")
print(f"   • Data coverage: {round((final_df.notna().sum().sum() / (len(final_df) * len(final_df.columns))) * 100, 1)}%")

print(f"\n🏠 BUILDING & UNIT STATISTICS:")
if 'num_buildings' in final_df.columns:
    print(f"   • Total buildings: {final_df['num_buildings'].sum():,.0f}")
    print(f"   • Parcels with buildings: {(final_df['num_buildings'] > 0).sum():,} ({((final_df['num_buildings'] > 0).sum() / len(final_df) * 100):.1f}%)")
    print(f"   • Average buildings per parcel: {final_df['num_buildings'].mean():.2f}")

if 'num_units' in final_df.columns:
    print(f"   • Total units: {final_df['num_units'].sum():,.0f}")
    print(f"   • Parcels with units: {(final_df['num_units'] > 0).sum():,} ({((final_df['num_units'] > 0).sum() / len(final_df) * 100):.1f}%)")
    print(f"   • Average units per parcel: {final_df['num_units'].mean():.2f}")

print(f"\n🏘️ GEOGRAPHIC DISTRIBUTION:")
if 'municipio' in final_df.columns:
    municipality_counts = final_df['municipio'].value_counts()
    print(f"   • Municipalities covered: {municipality_counts.nunique()}")
    print(f"   • Top municipalities:")
    for i, (muni, count) in enumerate(municipality_counts.head(3).items()):
        print(f"     {i+1}. {muni}: {count:,} parcels")

if 'provincia' in final_df.columns:
    province_counts = final_df['provincia'].value_counts()
    print(f"   • Provinces covered: {province_counts.nunique()}")

print(f"\n🏗️ BUILDING CHARACTERISTICS:")
if 'primary_building_type' in final_df.columns:
    building_types = final_df['primary_building_type'].value_counts()
    print(f"   • Building types identified: {building_types.nunique()}")
    for bt, count in building_types.head(3).items():
        print(f"     • {bt}: {count:,} parcels")

if 'primary_use_type' in final_df.columns:
    use_types = final_df['primary_use_type'].value_counts()
    print(f"   • Usage types identified: {use_types.nunique()}")
    for ut, count in use_types.head(3).items():
        print(f"     • {ut}: {count:,} parcels")

print(f"\n📏 AREA STATISTICS:")
if 'superficie_parcela' in final_df.columns:
    print(f"   • Total land area: {final_df['superficie_parcela'].sum():,.0f} sqm")
    print(f"   • Average parcel size: {final_df['superficie_parcela'].mean():.0f} sqm")
    print(f"   • Largest parcel: {final_df['superficie_parcela'].max():,.0f} sqm")

if 'total_constructed_area' in final_df.columns:
    total_constructed = final_df['total_constructed_area'].sum()
    print(f"   • Total constructed area: {total_constructed:,.0f} sqm")

print(f"\n🔍 SEARCH-READY FEATURES:")
search_features = [
    '✓ Geographic filtering (municipio, provincia)',
    '✓ Area-based searches (parcel size, built area, floor area)',
    '✓ Usage type filtering (residential, industrial, etc.)',
    '✓ Development intensity metrics (coverage ratios, density)',
    '✓ Building characteristics (age, type, count)',
    '✓ Utilization scoring (0-1 scale for development potential)'
]
for feature in search_features:
    print(f"   {feature}")

print(f"\n📁 EXCEL FILE READY:")
print(f"   📂 File: {excel_filename}")
print(f"   📋 Best sheet for search: 'Search_Ready_Data'")
print(f"   🎯 Contains {len([col for col in final_df.columns if col in ['referencia_catastral', 'superficie_parcela', 'municipio', 'num_buildings', 'num_units', 'primary_use_type', 'utilization_score']])} key search columns")

print(f"\n🚀 NEXT STEPS FOR SEARCH SYSTEM:")
print(f"   1. Import 'Search_Ready_Data' sheet into your search database")
print(f"   2. Index key columns: municipio, primary_use_type, superficie_parcela")
print(f"   3. Create range filters for: area, utilization_score, building counts")
print(f"   4. Use Municipality_Summary for region-level insights")

print("=" * 60)
print("✅ Ready to build your catastro search system!")
print("=" * 60)


🏗️  CATASTRO DATA PROCESSING COMPLETE

📊 DATASET OVERVIEW:
   • Total parcels processed: 10,110
   • Total columns created: 54
   • Data coverage: 75.5%

🏠 BUILDING & UNIT STATISTICS:
   • Total buildings: 6,495
   • Parcels with buildings: 1,982 (19.6%)
   • Average buildings per parcel: 0.64
   • Total units: 7,924
   • Parcels with units: 4,977 (49.2%)
   • Average units per parcel: 0.78

🏘️ GEOGRAPHIC DISTRIBUTION:
   • Municipalities covered: 2
   • Top municipalities:
     1. SELVA: 5,131 parcels
     2. SANTA MARIA DEL CAMI: 4,979 parcels
   • Provinces covered: 1

🏗️ BUILDING CHARACTERISTICS:
   • Building types identified: 15
     • No Buildings: 8,128 parcels
     • VIVIENDA: 1,002 parcels
     • ALMACEN: 518 parcels
   • Usage types identified: 15
     • No Units: 5,133 parcels
     • Agrario: 2,601 parcels
     • Residencial: 1,880 parcels

📏 AREA STATISTICS:
   • Total land area: 85,292,442 sqm
   • Average parcel size: 8436 sqm
   • Largest parcel: 4,102,279 sqm
   • Tota